In [1]:
import zipfile
import os

zip_path = "VUEFR_Payslips_VUE_V06_F0_Y2025_P04_VUEFR_DETAILLE_2.zip"
extract_path = "data/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete.")


Extraction complete.


In [9]:
!pip install pdfplumber pandas openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 5.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 5.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 5.6 MB/s eta 0:00:00a 0:00:01


In [10]:
import pdfplumber
import pandas as pd

pdf_path = "data/VUEFR_Payslips_VUE_V06_F0_Y2025_P04_VUEFR_DETAILLE_2.pdf"
output_path = "data/page_table.xlsx"
page_number = 2  # 0-indexed, so page 3 = 2

with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[page_number]
    table = page.extract_table()

# Convert to DataFrame
df = pd.DataFrame(table[1:], columns=table[0])
df.to_excel(output_path, index=False)

print(f"✅ Table from page {page_number+1} saved to {output_path}")


✅ Table from page 3 saved to data/page_table.xlsx


In [15]:
import pdfplumber
import pandas as pd

pdf_path = "data/VUEFR_Payslips_VUE_V06_F0_Y2025_P04_VUEFR_DETAILLE_2.pdf"
page_number = 2  # for example, page 3 (0-indexed)
output_excel = "data/output_corrected.xlsx"

with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[page_number]
    
    # Extract table(s)
    tables = page.extract_tables()
    print(f"Number of tables found: {len(tables)}")

    # Pick the largest table (assuming it’s the main one)
    main_table = max(tables, key=lambda t: len(t))
    
    # Show a preview to inspect
    for row in main_table[:5]:
        print(row)


Number of tables found: 2
['Code', 'Libellé', 'Base', 'TauxSal', None, 'Montant Sal.', None, 'TauxPat', 'Montant Pat.']
['1010\n1191\n1192\n2003\n2011\n2407\n2409\n2503\n2623\n2624\n2625\n60G1\n6921\n7060\n7070\n70N0\n71P0\n71Q0\n7250\n7280\n7280\n7280\n7280\n72J0\n7300\n7310\n7320\n7900\n7920\n7930\n7950\n79P0\n79S0\n7D00\n7D10\n7S00\n7X01\n7X21\n7X41\n7X42', 'Salaire de Base\nPaiement sup. proratisé\nBonus responsabilité CPT\nBonus trimestriel\nRappel sur Salaire\nInternational Per Diem\nInternational Overnight P\nFormation et activité adm\nComplément de salaire PNT\nPlus base Ile-de-France\nPrime de produits assuran\nTOTAL BRUT\nURSSAF Maladie NRésident\nMedical insurance (MULTI)\nURSSAF Vieillesse\nURSSAF Vieillesse TA\nURSSAF Solidarité\nAlloc Fam. Tx Réd.\nURSSAF Alloc Fam. Tx Dif.\nURSSAF Acc Travail\nURSSAF Forfait social\nURSSAF Forfait social 0325\nURSSAF Forfait social 0225\nURSSAF Forfait social 0125\nURSSAF FNAL\nPôle Emploi TA\nPôle Emploi TB\nPôle Emploi AGS\nMutuelle - 

In [16]:
# Convert list of lists to DataFrame
df = pd.DataFrame(main_table[1:], columns=main_table[0])

# Optional: clean up column names (strip spaces)
df.columns = [col.strip() if col else f"col_{i}" for i, col in enumerate(df.columns)]

# Inspect the DataFrame
print(df.head())

# Save to Excel
df.to_excel(output_excel, index=False)
print(f"✅ Saved cleaned table to {output_excel}")


                                                Code  \
0  1010\n1191\n1192\n2003\n2011\n2407\n2409\n2503...   
1                                               /510   
2     NET À PAYER AVANT IMPÔT SUR LE REVENU 9.570,34   
3  dont évolution de la rémunération liée à la su...   
4                    Impôt sur le revenu à la source   

                                             Libellé  \
0  Salaire de Base\nPaiement sup. proratisé\nBonu...   
1                                 Montant net social   
2                                               None   
3                                               None   
4                                               None   

                                                Base  \
0  92,52\n10,00\n1,00\n4,00\n11.610,10\n11.610,10...   
1                                                      
2                                               None   
3                                               None   
4                                             

In [21]:
import pandas as pd

# Load your data
df = pd.read_excel("data/output_corrected.xlsx")

# Step 1: Split cells into lists
for col in df.columns:
    df[col] = df[col].astype(str).str.split('\n')

# Step 2: Pad each row's lists so they all have the same length
def pad_row(row):
    max_len = max(len(row[col]) for col in df.columns)
    for col in df.columns:
        row[col] += ['0'] * (max_len - len(row[col]))  # pad with '0' as string
    return row

df = df.apply(pad_row, axis=1)

# Step 3: Now explode all columns
df_expanded = df.explode(list(df.columns)).reset_index(drop=True)

# Step 4: Clean up (strip whitespace)
df_expanded = df_expanded.map(lambda x: x.strip() if isinstance(x, str) else x)

# Step 5: Replace any remaining empty cells with 0
df_expanded.replace('', '0', inplace=True)
df_expanded.fillna('0', inplace=True)

# Step 6: Save
output_path = "data/final_table_cleaned.xlsx"
df_expanded.to_excel(output_path, index=False)

print(f"✅ All done! Final table saved to: {output_path}")

✅ All done! Final table saved to: data/final_table_cleaned.xlsx
